In [3]:
import pandas as pd
import numpy as np

import re
from itertools import chain
from collections import Counter

from google.cloud import bigquery
from google.oauth2 import service_account

from datetime import datetime, timedelta

import requests
import warnings
from requests.packages.urllib3.exceptions import InsecureRequestWarning

# InsecureRequestWarning 경고 무시
warnings.simplefilter('ignore', InsecureRequestWarning)

In [4]:
KEY_PATH = ".config/"
servicekey_path = KEY_PATH + "serviceKey.json" ## 빅쿼리 외 다른 API 활용 위해
bigquerykey_path = KEY_PATH + "mido-project-426906-31b49963ac97.json"

warnings.filterwarnings("ignore")

In [5]:
# BigQuery 클라이언트 생성 함수
def create_bigquery_client(key_path):
    credentials = service_account.Credentials.from_service_account_file(key_path)
    client = bigquery.Client(credentials=credentials, project=credentials.project_id)
    return client

In [6]:
def save_dataframe_to_bigquery(df, dataset_id, table_id, key_path):
    # BigQuery 클라이언트 객체 생성
    client = create_bigquery_client(key_path)

    # 테이블 레퍼런스 생성
    table_ref = client.dataset(dataset_id).table(table_id)

    # 데이터프레임을 BigQuery 테이블에 적재
    job_config = bigquery.LoadJobConfig()
    job_config.write_disposition = "WRITE_TRUNCATE"  # 기존 테이블 내용 삭제 후 삽입

    job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
    job.result()  # 작업 완료 대기

    print(f"Data inserted into table {table_id} successfully.")

In [7]:
def get_dataframe_from_bigquery(dataset_id, table_id, key_path):
    # BigQuery 클라이언트 생성
    client = create_bigquery_client(key_path)

    # 테이블 레퍼런스 생성
    table_ref = client.dataset(dataset_id).table(table_id)

    # 테이블 데이터를 DataFrame으로 변환
    df = client.list_rows(table_ref).to_dataframe()

    return df

In [8]:
# 오늘 날짜
today = datetime.today()#.strftime('%Y%m%d')

# 어제 날짜 계산
ytday = datetime.today() - timedelta(days=1)

# 만약 어제, 오늘이 토요일(5) 또는 일요일(6)이라면, 그 전주 금요일로 변경
if ytday.weekday() == 5:  # 토요일
    ytday -= timedelta(days=1)
elif ytday.weekday() == 6:  # 일요일
    ytday -= timedelta(days=2)
if today.weekday() == 5:  # 토요일
    today -= timedelta(days=1)
elif today.weekday() == 6:  # 일요일
    today -= timedelta(days=2)

# 'YYYYMMDD' 형식으로 변환
ytday = ytday.strftime('%Y%m%d')
today = today.strftime('%Y%m%d')

#### 법정동코드

In [9]:
dist_code = pd.read_csv('C:\py_src\midoproject\data/법정동코드 전체자료.csv',encoding='cp949')

In [10]:
dist_code_list = []
for i in range(dist_code['법정동명'].str.split().apply(len).max()):
    dist_code_temp = dist_code[dist_code['법정동명'].str.split().str[i].str[-1].isin(['읍','면','동','리','가','로']).dropna()]
    dist_code_list.append([dist_code_temp['법정동명'].str.split().str[i].dropna().unique()])

emd_code_list = list(chain(*chain(*dist_code_list)))

#### 종합쇼핑몰 납품상세내역

In [11]:
all_shop_df = get_dataframe_from_bigquery('g2b', 'shop_detail_df_all', bigquerykey_path)

In [12]:
all_shop_df_fin = all_shop_df[['납품요구접수일자', '수요기관명', '납품요구건명', '업체명', '단가', '단위', '수량', '금액', '수요기관코드', '수요기관구분', '수요기관지역명','납품요구지청명']]
all_shop_df_fin = all_shop_df_fin.sort_values(['납품요구접수일자'],ascending=False).reset_index(drop=True)

In [13]:
# 특수문자, 숫자, 영어 제거 함수 (연속된 공백을 단일 공백으로 변환 포함)
def clean_text(text):
    text = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣\s]', ' ', text)
    text = re.sub('\s+', ' ', text).strip()  # 연속된 공백을 단일 공백으로 변환
    return text

In [14]:
# 사업명 전처리
all_shop_df_fin['납품요구건명_re'] = all_shop_df_fin['납품요구건명'].apply(clean_text)
# all_shop_df_fin['납품요구건명_re'] = all_shop_df_fin['납품요구건명_re'].str.replace('초등학교','').str.replace('중학교','').str.replace('고등학교','')
all_shop_df_fin['납품요구건명_re'] = all_shop_df_fin['납품요구건명_re'].str.strip()

all_shop_df_fin['납품요구건명_re'] = all_shop_df_fin['납품요구건명_re'].str.split().apply(lambda x: [kw for kw in x if len(kw) > 1]) ## 글자수 1 제외
all_shop_df_fin['납품요구건명_re'] = all_shop_df_fin['납품요구건명_re'].apply((' ').join)

In [15]:
# 수요기관 필터링
dist_nm = pd.Series(all_shop_df_fin[all_shop_df_fin['수요기관지역명'].str.split(' ').str[1].notnull()]['수요기관지역명'].str.split(' ').str[1].unique())
dist_nm = np.where(dist_nm.apply(len)<=2,dist_nm,dist_nm.str[:-1])

#### 지자체 세부사업별 예산서

In [20]:
test = pd.read_csv('C:\py_src\midoproject\data/2024-07-10T00-34_export.csv')

In [21]:
# 빅쿼리에서 불러오기
budget_df_today = get_dataframe_from_bigquery('budget','budget_df_0' + today,bigquerykey_path)
# budget_df_ytday = get_dataframe_from_bigquery('budget','budget_df_0' + ytday,bigquerykey_path)

In [22]:
budget_df_today = budget_df_today[budget_df_today['세부사업명'].isin(test['세부사업명'].unique())].reset_index(drop=True)

In [24]:
# 필요키워드1
need_kwd1 = '학교|초등학교|중학교|고등학교|대학교|다목적|운동장|종합운동장|공원|체육공원|체육시설|체육센터|스포츠센터|연병장|구장|축구장|족구장|야구장|풋살장|경기장|배드민턴장|테니스장|게이트볼장|하키장|관급'

# 필요키워드2
need_kwd2 = '개선|교체|보수|공사|구매|구입|설치|정비|조성'

# 필요키워드3
need_kwd3 = '인조잔디'

# 제외키워드
except_kwd = 'CCTV|CCTV|LED|가로등|가속기|간접등|간판|개방|검진|경관|계단|골프장|관리|광고|교량|교실|교육|교차로|균열|그늘막|그림|급수관|급식|난방기|내진|냉방기|네트워크|노상|노점|농기계|단열|데이터|도로개설|도로방음벽|도로정비|도로확장|도서관|디자인|뚜껑|막구조물|맨홀|모노레일|모니터링|미끄럼|미세먼지|미술|방송|방수|방음|방음벽|배면도로|배수|버스|법률자문|벤치|벽화|보건|보도블럭|보상|보일러|보행자도로|복지|볼라드|분묘|불법|빗물받이|사용료|사워실|살포|생육환경|석면|선홈통|소프트웨어|수도|수로|수목표찰|수영장|스크린|스프링클러|승강기|시계탑|시스템|식수대|신호등|안심벨|안전|약수터|억제제|역량|연결도로|예방|예술|오염|옹벽|요양기관|운영|울타리|위생|위험수목|유아|육교|음성안내|음악|의자|인양기|인터넷|입학|자료|자전거|저장고|저장장치|전광판|전기|전신주|전통시장|정화|정화조|조도|조명|조형물|조화기|주차장|줄눈|진로진학|진입도로|창고|창출|처우|철거|취업|치안|카메라|카약|통학로|파고라|펜스|폐기물|폐열|포충기|풀장|풍차|하수|하수관로|하수도|하천|학습|행사|화장|화장실|활성화|횡단보도|휀스|흡연'

In [25]:
budget_df_today = budget_df_today[((budget_df_today['세부사업명'].str.contains(need_kwd1) & 
                                  budget_df_today['세부사업명'].str.contains(need_kwd2)) &
                                  ~budget_df_today['세부사업명'].str.contains(except_kwd)) |
                                  budget_df_today['세부사업명'].str.contains(need_kwd3)
                                  ].reset_index(drop=True)

In [26]:
# 사업명 전처리
budget_df_today['세부사업명_re'] = budget_df_today['세부사업명'].apply(clean_text)
# budget_df_today['세부사업명_re'] = budget_df_today['세부사업명_re'].str.replace('초등학교','').str.replace('중학교','').str.replace('고등학교','')
budget_df_today['세부사업명_re'] = budget_df_today['세부사업명_re'].str.strip()

budget_df_today['세부사업명_re'] = budget_df_today['세부사업명_re'].str.split().apply(lambda x: [kw for kw in x if len(kw) > 1]) ## 글자수 1 제외
budget_df_today['세부사업명_re'] = budget_df_today['세부사업명_re'].apply((' ').join)

In [27]:
# 해당 지역 추출
filtered_budget_df_temp1 = budget_df_today[budget_df_today['자치단체명'].str.contains(('|').join(dist_nm))].reset_index(drop=True)
filtered_budget_df_temp2 = budget_df_today[budget_df_today['자치단체명'].str.contains('본청')].reset_index(drop=True)

filtered_budget_df_today = pd.concat([filtered_budget_df_temp1,filtered_budget_df_temp2],axis=0).reset_index(drop=True)

#### 지자체 종합쇼핑몰 전처리

In [28]:
# Initialize the dictionary for split keywords
shop_split_kwd = {}
for i in range(all_shop_df_fin['납품요구건명_re'].str.split().apply(len).max()):
    shop_split_kwd['kwd_' + str(i)] = all_shop_df_fin['납품요구건명_re'].str.split().str[i]

In [29]:
# 매핑데이터셋
mapping_merge_df = []

In [30]:
# 종합쇼핑몰 납품요구건명 split 키워드 순서대로 맵핑
# Initialize the list to store the results
prep_budget_df_today = filtered_budget_df_today.copy()
prep_all_shop_df_fin = all_shop_df_fin.copy()

# Iterate over each keyword column
for i in range(len(shop_split_kwd)):
    keyword_column = 'kwd_' + str(i)
    
    # Get unique non-null keywords
    shop_split_kwd_temp = shop_split_kwd[keyword_column].dropna().unique()

    prep_budget_df_today[keyword_column] = prep_budget_df_today['세부사업명_re'].apply(lambda x: ' '.join(re.findall('|'.join(shop_split_kwd_temp), x)))
    prep_budget_df_today[keyword_column] = prep_budget_df_today[keyword_column].str.strip()
    prep_budget_df_today[keyword_column] = prep_budget_df_today[keyword_column].str.split().apply(lambda x : ' '.join(sorted(set(x)))).str.strip()

    prep_all_shop_df_fin[keyword_column] = prep_all_shop_df_fin['납품요구건명_re'].apply(lambda x: ' '.join(re.findall('|'.join(shop_split_kwd_temp), x)))
    prep_all_shop_df_fin[keyword_column] = prep_all_shop_df_fin[keyword_column].str.strip()
    prep_all_shop_df_fin[keyword_column] = prep_all_shop_df_fin[keyword_column].str.split().apply(lambda x : ' '.join(sorted(set(x)))).str.strip()

    mapping_merge_df_temp = pd.merge(
            prep_budget_df_today[(prep_budget_df_today[keyword_column] != '') & ~(prep_budget_df_today[keyword_column].isnull())][['지역명','자치단체명','세부사업코드','세부사업명', '세부사업명_re', keyword_column]].reset_index(drop=True),
            prep_all_shop_df_fin[(prep_all_shop_df_fin[keyword_column] != '') & ~(prep_all_shop_df_fin[keyword_column].isnull())][['수요기관명','수요기관지역명','납품요구건명', '납품요구건명_re', keyword_column]].reset_index(drop=True),
            how='left',
            on=keyword_column
            ).drop_duplicates().reset_index(drop=True)
    
    # Append the temporary DataFrame to the results list
    mapping_merge_df.append(mapping_merge_df_temp)

In [31]:
# 종합쇼핑몰 지역 키워드 맵핑 - 세부사업명 지역 및 해당지역 추출
# Initialize the columns for district keywords
prep_budget_df_today = filtered_budget_df_today.copy()
prep_all_shop_df_fin = all_shop_df_fin.copy()

sig_pat = '|'.join(dist_nm)

prep_budget_df_today['kwd_sig'] = prep_budget_df_today['세부사업명_re'].apply(lambda x: ' '.join(re.findall(sig_pat, x)))
prep_budget_df_today['kwd_sig'] = prep_budget_df_today['kwd_sig'].str.strip()
prep_budget_df_today['kwd_sig'] = prep_budget_df_today['kwd_sig'].str.split()
prep_budget_df_today = prep_budget_df_today.explode('kwd_sig')

prep_all_shop_df_fin['kwd_sig'] = prep_all_shop_df_fin['납품요구건명_re'].apply(lambda x: ' '.join(re.findall(sig_pat, x)))
prep_all_shop_df_fin['kwd_sig'] = prep_all_shop_df_fin['kwd_sig'].str.strip()
prep_all_shop_df_fin['kwd_sig'] = prep_all_shop_df_fin['kwd_sig'].str.split()
prep_all_shop_df_fin = prep_all_shop_df_fin.explode('kwd_sig')

mapping_merge_df_sig = pd.merge(
    prep_budget_df_today[(prep_budget_df_today['kwd_sig'] != '') & ~(prep_budget_df_today['kwd_sig'].isnull())][['지역명','자치단체명','세부사업코드','세부사업명', '세부사업명_re', 'kwd_sig']].reset_index(drop=True),
    prep_all_shop_df_fin[(prep_all_shop_df_fin['kwd_sig'] != '') & ~(prep_all_shop_df_fin['kwd_sig'].isnull())][['수요기관명','수요기관지역명','납품요구건명', '납품요구건명_re', 'kwd_sig']].reset_index(drop=True),
    how='left',
    on='kwd_sig'
    ).drop_duplicates().reset_index(drop=True)

# Append the district keywords DataFrame to the results list
mapping_merge_df.append(mapping_merge_df_sig)

In [32]:
# 종합쇼핑몰 지역 키워드 맵핑 - 세부사업명 지역 및 해당지역 읍면동 추출
# Initialize the columns for district keywords
prep_budget_df_today = filtered_budget_df_today.copy()
prep_all_shop_df_fin = all_shop_df_fin.copy()

emd_pat = '|'.join(set(emd_code_list))

prep_budget_df_today['kwd_emd'] = prep_budget_df_today['세부사업명_re'].apply(lambda x: ' '.join(re.findall(emd_pat, x)))
prep_budget_df_today['kwd_emd'] = prep_budget_df_today['kwd_emd'].str.strip()
prep_budget_df_today['kwd_emd'] = prep_budget_df_today['kwd_emd'].str.split()
prep_budget_df_today = prep_budget_df_today.explode('kwd_emd')

prep_all_shop_df_fin['kwd_emd'] = prep_all_shop_df_fin['납품요구건명_re'].apply(lambda x: ' '.join(re.findall(emd_pat, x)))
prep_all_shop_df_fin['kwd_emd'] = prep_all_shop_df_fin['kwd_emd'].str.strip()
prep_all_shop_df_fin['kwd_emd'] = prep_all_shop_df_fin['kwd_emd'].str.split()
prep_all_shop_df_fin = prep_all_shop_df_fin.explode('kwd_emd')

mapping_merge_df_emd = pd.merge(
    prep_budget_df_today[(prep_budget_df_today['kwd_emd'] != '') & ~(prep_budget_df_today['kwd_emd'].isnull())][['지역명','자치단체명','세부사업코드','세부사업명', '세부사업명_re', 'kwd_emd']].reset_index(drop=True),
    prep_all_shop_df_fin[(prep_all_shop_df_fin['kwd_emd'] != '') & ~(prep_all_shop_df_fin['kwd_emd'].isnull())][['수요기관명','수요기관지역명','납품요구건명', '납품요구건명_re', 'kwd_emd']].reset_index(drop=True),
    how='left',
    on='kwd_emd'
    ).drop_duplicates().reset_index(drop=True)

# Append the district keywords DataFrame to the results list
mapping_merge_df.append(mapping_merge_df_emd)

In [33]:
# 필요키워드 앞부분 추출
need_kwd_split = list(set(chain(*all_shop_df_fin['납품요구건명_re'].apply(lambda x: re.split(need_kwd1,x)).apply(''.join).str.split())))
need_kwd_split_list = list(pd.Series(need_kwd_split)[pd.Series(need_kwd_split).apply(len) >= 2])

In [34]:
# 종합쇼핑몰 지역 키워드 맵핑 - 필요키워드 앞부분 매핑
# Initialize the columns for district keywords
prep_budget_df_today = filtered_budget_df_today.copy()
prep_all_shop_df_fin = all_shop_df_fin.copy()

kwd_split_pat = '|'.join(set(need_kwd_split_list))

prep_budget_df_today['kwd_need_split'] = prep_budget_df_today['세부사업명_re'].apply(lambda x: ' '.join(re.findall(kwd_split_pat, x)))
prep_budget_df_today['kwd_need_split'] = prep_budget_df_today['kwd_need_split'].str.strip()
prep_budget_df_today['kwd_need_split'] = prep_budget_df_today['kwd_need_split'].str.split()
prep_budget_df_today = prep_budget_df_today.explode('kwd_need_split')

prep_all_shop_df_fin['kwd_need_split'] = prep_all_shop_df_fin['납품요구건명_re'].apply(lambda x: ' '.join(re.findall(kwd_split_pat, x)))
prep_all_shop_df_fin['kwd_need_split'] = prep_all_shop_df_fin['kwd_need_split'].str.strip()
prep_all_shop_df_fin['kwd_need_split'] = prep_all_shop_df_fin['kwd_need_split'].str.split()
prep_all_shop_df_fin = prep_all_shop_df_fin.explode('kwd_need_split')

mapping_merge_df_need_split = pd.merge(
    prep_budget_df_today[(prep_budget_df_today['kwd_need_split'] != '') & ~(prep_budget_df_today['kwd_need_split'].isnull())][['지역명','자치단체명','세부사업코드','세부사업명', '세부사업명_re', 'kwd_need_split']].reset_index(drop=True),
    prep_all_shop_df_fin[(prep_all_shop_df_fin['kwd_need_split'] != '') & ~(prep_all_shop_df_fin['kwd_need_split'].isnull())][['수요기관명','수요기관지역명','납품요구건명', '납품요구건명_re', 'kwd_need_split']].reset_index(drop=True),
    how='left',
    on='kwd_need_split'
    ).drop_duplicates().reset_index(drop=True)

# Append the district keywords DataFrame to the results list
mapping_merge_df.append(mapping_merge_df_need_split)

In [35]:
# 종합쇼핑몰 지역 키워드 맵핑 - 필요키워드 매핑
# Initialize the columns for district keywords
prep_budget_df_today = filtered_budget_df_today.copy()
prep_all_shop_df_fin = all_shop_df_fin.copy()

prep_budget_df_today['kwd_need'] = prep_budget_df_today['세부사업명_re'].apply(lambda x: ' '.join(re.findall(need_kwd1, x)))
prep_budget_df_today['kwd_need'] = prep_budget_df_today['kwd_need'].str.strip()
prep_budget_df_today['kwd_need'] = prep_budget_df_today['kwd_need'].str.split()
prep_budget_df_today = prep_budget_df_today.explode('kwd_need')

prep_all_shop_df_fin['kwd_need'] = prep_all_shop_df_fin['납품요구건명_re'].apply(lambda x: ' '.join(re.findall(need_kwd1, x)))
prep_all_shop_df_fin['kwd_need'] = prep_all_shop_df_fin['kwd_need'].str.strip()
prep_all_shop_df_fin['kwd_need'] = prep_all_shop_df_fin['kwd_need'].str.split()
prep_all_shop_df_fin = prep_all_shop_df_fin.explode('kwd_need')

mapping_merge_df_need = pd.merge(
    prep_budget_df_today[(prep_budget_df_today['kwd_need'] != '') & ~(prep_budget_df_today['kwd_need'].isnull())][['지역명','자치단체명','세부사업코드','세부사업명', '세부사업명_re', 'kwd_need']].reset_index(drop=True),
    prep_all_shop_df_fin[(prep_all_shop_df_fin['kwd_need'] != '') & ~(prep_all_shop_df_fin['kwd_need'].isnull())][['수요기관명','수요기관지역명','납품요구건명', '납품요구건명_re', 'kwd_need']].reset_index(drop=True),
    how='left',
    on='kwd_need'
    ).drop_duplicates().reset_index(drop=True)

# Append the district keywords DataFrame to the results list
mapping_merge_df.append(mapping_merge_df_need)

In [36]:
# 키워드별 데이터셋 병합
mapping_merge_df_fin = pd.concat(mapping_merge_df, axis=0).drop_duplicates()
mapping_merge_df_fin = mapping_merge_df_fin[~mapping_merge_df_fin['납품요구건명'].isnull()].reset_index(drop=True)

In [37]:
# 모든 키워드 합
kwd_columns = ['kwd_0','kwd_1','kwd_2','kwd_3','kwd_4','kwd_5','kwd_6','kwd_7','kwd_8','kwd_sig','kwd_emd','kwd_need_split','kwd_need']
mapping_merge_df_fin['all_kwd'] = mapping_merge_df_fin[kwd_columns].apply(lambda x : (' ').join(x.dropna()),axis=1)
mapping_merge_df_fin['all_kwd'] = mapping_merge_df_fin['all_kwd'].str.strip()
mapping_merge_df_fin['all_kwd'] = mapping_merge_df_fin['all_kwd'].str.split().apply(lambda x : ' '.join(sorted(set(x)))).str.strip()
mapping_merge_df_fin.drop(columns=kwd_columns, inplace=True)

In [38]:
# 지역명 추출1
mapping_merge_df_fin['수요기관지역명_split1'] = mapping_merge_df_fin['수요기관지역명'].str.split().str[0]

# Define the replacements
replacements = {
    '전라북도': '전북',
    '전라남도': '전남',
    '경상북도': '경북',
    '경상남도': '경남',
    '충청북도': '충북',
    '충청남도': '충남'
}

# Apply the replacements
mapping_merge_df_fin['수요기관지역명_split1'] = mapping_merge_df_fin['수요기관지역명_split1'].replace(replacements)
mapping_merge_df_fin['수요기관지역명_split1'] = mapping_merge_df_fin['수요기관지역명_split1'].str[:2]

In [39]:
# 지역명 추출2
mapping_merge_df_fin['수요기관지역명_split2'] = mapping_merge_df_fin['수요기관지역명'].str.split().str[1]
dist_sig_pat = '|'.join(mapping_merge_df_fin['수요기관지역명_split2'].dropna().unique())

mapping_merge_df_fin['자치단체명_re'] = mapping_merge_df_fin['자치단체명'].apply(lambda x: ' '.join(re.findall(dist_sig_pat, x)))
mapping_merge_df_fin['자치단체명_re'] = mapping_merge_df_fin['자치단체명_re'].str.strip()
mapping_merge_df_fin['자치단체명_re'] = mapping_merge_df_fin['자치단체명_re'].str.split()
mapping_merge_df_fin = mapping_merge_df_fin.explode('자치단체명_re')

In [40]:
# 제외 지역 제거
filtered_mapping_merge_df_fin = mapping_merge_df_fin[mapping_merge_df_fin['자치단체명_re']!=''].drop_duplicates().reset_index(drop=True)

In [41]:
# 지역 일치 확인1
mapping_merge_df_fin_temp1 = filtered_mapping_merge_df_fin[filtered_mapping_merge_df_fin['지역명']==filtered_mapping_merge_df_fin['수요기관지역명_split1']].drop_duplicates().reset_index(drop=True)
mapping_merge_df_fin_temp1 = mapping_merge_df_fin_temp1[mapping_merge_df_fin_temp1['자치단체명'].str.contains('|'.join(mapping_merge_df_fin_temp1['수요기관지역명_split2'].dropna().unique()))].drop_duplicates().reset_index(drop=True)
mapping_merge_df_fin_temp1 = mapping_merge_df_fin_temp1[mapping_merge_df_fin_temp1['자치단체명_re']==mapping_merge_df_fin_temp1['수요기관지역명_split2']].drop_duplicates().reset_index(drop=True)

# 본청 포함 확인
mapping_merge_df_fin_temp2 = filtered_mapping_merge_df_fin[filtered_mapping_merge_df_fin['자치단체명'].str.contains('본청')].drop_duplicates().reset_index(drop=True)

# 데이터 병합
mapping_merge_df_final = pd.concat([mapping_merge_df_fin_temp1,mapping_merge_df_fin_temp2], axis=0).drop_duplicates().reset_index(drop=True)

In [42]:
# 그룹화할 컬럼 리스트
group_cols = ['지역명', '자치단체명', '세부사업코드', '세부사업명', '세부사업명_re', '수요기관명', '수요기관지역명', '납품요구건명', '납품요구건명_re','수요기관지역명_split1', '자치단체명_re', '수요기관지역명_split2']

# 그룹화 및 all_kwd 컬럼 조인
mapping_merge_df_final_grp = mapping_merge_df_final.groupby(group_cols).agg({'all_kwd': ' '.join}).reset_index()
mapping_merge_df_final_grp['all_kwd'] = mapping_merge_df_final_grp['all_kwd'].str.strip()
mapping_merge_df_final_grp['all_kwd'] = mapping_merge_df_final_grp['all_kwd'].str.split().apply(lambda x : ' '.join(sorted(set(x)))).str.strip()

In [43]:
# 필요키워드 기준
mapping_merge_df_final_grp['세부사업명_키워드1'] = mapping_merge_df_final_grp['세부사업명'].apply(lambda x: ' '.join(re.findall(need_kwd1, x)))
mapping_merge_df_final_grp['납품요구건명_키워드1'] = mapping_merge_df_final_grp['납품요구건명'].apply(lambda x: ' '.join(re.findall(need_kwd1, x)))

# 사업명 지역 추출
mapping_merge_df_final_grp['세부사업명_지역'] = mapping_merge_df_final_grp['세부사업명'].apply(lambda x: ' '.join(re.findall(emd_pat, x)))
mapping_merge_df_final_grp['납품요구건명_지역'] = mapping_merge_df_final_grp['납품요구건명'].apply(lambda x: ' '.join(re.findall(emd_pat, x)))

# 필요키워드 패턴 기준
need_kwd_pat = '\w+' + ('|\w+').join(need_kwd1.split('|'))

mapping_merge_df_final_grp['세부사업명_키워드2'] = mapping_merge_df_final_grp['세부사업명'].apply(lambda x: ' '.join(re.findall(need_kwd_pat, x)))
mapping_merge_df_final_grp['납품요구건명_키워드2'] = mapping_merge_df_final_grp['납품요구건명'].apply(lambda x: ' '.join(re.findall(need_kwd_pat, x)))

In [44]:
def check_inclusion1(row):
    set1 = set(row['세부사업명_키워드1'])
    set2 = set(row['납품요구건명_키워드1'])
    return bool(set1 & set2)

In [45]:
# 필요키워드 모두 없는것 제외
mapping_merge_df_final_grp_temp1 = mapping_merge_df_final_grp[(mapping_merge_df_final_grp['세부사업명_키워드1']!='')&(mapping_merge_df_final_grp['납품요구건명_키워드1']!='')].reset_index(drop=True)

# 필요키워드 포함여부 확인
mapping_merge_df_final_grp_temp1['세부사업명_키워드1'] = mapping_merge_df_final_grp_temp1['세부사업명_키워드1'].str.split() ## 키워드별 정리
mapping_merge_df_final_grp_temp1['납품요구건명_키워드1'] = mapping_merge_df_final_grp_temp1['납품요구건명_키워드1'].str.split() ## 키워드별 정리

mapping_merge_df_final_grp_temp1 = mapping_merge_df_final_grp_temp1[mapping_merge_df_final_grp_temp1.apply(check_inclusion1,axis=1)].reset_index(drop=True)

mapping_merge_df_final_grp_temp1['세부사업명_키워드1'] = mapping_merge_df_final_grp_temp1['세부사업명_키워드1'].apply(lambda x : ' '.join(sorted(set(x)))).str.strip()
mapping_merge_df_final_grp_temp1['납품요구건명_키워드1'] = mapping_merge_df_final_grp_temp1['납품요구건명_키워드1'].apply(lambda x : ' '.join(sorted(set(x)))).str.strip()

In [46]:
def check_inclusion_dist(row):
    set1 = set(row['세부사업명_지역'])
    set2 = set(row['납품요구건명_지역'])
    return bool(set1 & set2)

In [47]:
# 지역 없는것 제외
mapping_merge_df_final_grp_temp2 = mapping_merge_df_final_grp[(mapping_merge_df_final_grp['세부사업명_지역']!='')&(mapping_merge_df_final_grp['납품요구건명_지역']!='')].reset_index(drop=True)

# 사업명 지역 포함여부 확인
mapping_merge_df_final_grp_temp2['세부사업명_지역'] = mapping_merge_df_final_grp_temp2['세부사업명_지역'].str.split()
mapping_merge_df_final_grp_temp2['납품요구건명_지역'] = mapping_merge_df_final_grp_temp2['납품요구건명_지역'].str.split()

mapping_merge_df_final_grp_temp2 = mapping_merge_df_final_grp_temp2[mapping_merge_df_final_grp_temp2.apply(check_inclusion_dist,axis=1)].reset_index(drop=True)

mapping_merge_df_final_grp_temp2['세부사업명_지역'] = mapping_merge_df_final_grp_temp2['세부사업명_지역'].apply(lambda x : ' '.join(sorted(set(x)))).str.strip() ## 키워드별 정리
mapping_merge_df_final_grp_temp2['납품요구건명_지역'] = mapping_merge_df_final_grp_temp2['납품요구건명_지역'].apply(lambda x : ' '.join(sorted(set(x)))).str.strip() ## 키워드별 정리

In [48]:
def check_inclusion2(row):
    set1 = set(row['세부사업명_키워드2'])
    set2 = set(row['납품요구건명_키워드2'])
    return bool(set1 & set2)

In [49]:
# 필요키워드패턴 모두 없는것 제외
mapping_merge_df_final_grp_temp3 = mapping_merge_df_final_grp[(mapping_merge_df_final_grp['세부사업명_키워드2']!='')&(mapping_merge_df_final_grp['납품요구건명_키워드2']!='')].reset_index(drop=True)

# 필요키워드 패턴 포함여부 확인
mapping_merge_df_final_grp_temp3['세부사업명_키워드2'] = mapping_merge_df_final_grp_temp3['세부사업명_키워드2'].str.split() ## 키워드별 정리
mapping_merge_df_final_grp_temp3['납품요구건명_키워드2'] = mapping_merge_df_final_grp_temp3['납품요구건명_키워드2'].str.split() ## 키워드별 정리

mapping_merge_df_final_grp_temp3 = mapping_merge_df_final_grp_temp3[mapping_merge_df_final_grp_temp3.apply(check_inclusion2,axis=1)].reset_index(drop=True)

mapping_merge_df_final_grp_temp3['세부사업명_키워드2'] = mapping_merge_df_final_grp_temp3['세부사업명_키워드2'].apply(lambda x : ' '.join(sorted(set(x)))).str.strip()
mapping_merge_df_final_grp_temp3['납품요구건명_키워드2'] = mapping_merge_df_final_grp_temp3['납품요구건명_키워드2'].apply(lambda x : ' '.join(sorted(set(x)))).str.strip()

In [50]:
# 필터링 데이터셋 병합
mapping_merge_df_final_grp_fin = pd.concat([mapping_merge_df_final_grp_temp1,mapping_merge_df_final_grp_temp2,mapping_merge_df_final_grp_temp3]).drop_duplicates().reset_index(drop=True)

In [51]:
# 모든 키워드 합 중복 제거
kwd_columns = ['all_kwd','세부사업명_키워드1', '납품요구건명_키워드1', '세부사업명_지역', '납품요구건명_지역', '세부사업명_키워드2', '납품요구건명_키워드2']
mapping_merge_df_final_grp_fin['all_kwd_fin'] = mapping_merge_df_final_grp_fin[kwd_columns].apply(lambda x : (' ').join(x.dropna()),axis=1)
mapping_merge_df_final_grp_fin['all_kwd_fin'] = mapping_merge_df_final_grp_fin['all_kwd_fin'].str.strip()
mapping_merge_df_final_grp_fin['all_kwd_fin'] = mapping_merge_df_final_grp_fin['all_kwd_fin'].str.split().apply(lambda x : ' '.join(sorted(set(x)))).str.strip()
# mapping_merge_df_final_grp_fin.drop(columns=kwd_columns, inplace
# =True)

mapping_merge_df_final_grp_fin = mapping_merge_df_final_grp_fin.drop_duplicates(['지역명', '자치단체명', '세부사업코드', '세부사업명', '세부사업명_re', '수요기관명', '수요기관지역명',
                                                                                 '납품요구건명', '납품요구건명_re', '수요기관지역명_split1', '자치단체명_re', '수요기관지역명_split2', 'all_kwd_fin']).reset_index(drop=True)

In [52]:
# 최종필터링1 - 지역 일치
mapping_merge_df_final_grp_fin_temp1 = mapping_merge_df_final_grp_fin[(mapping_merge_df_final_grp_fin['세부사업명_지역']!='') & (mapping_merge_df_final_grp_fin['납품요구건명_지역']!='')]
mapping_merge_df_final_grp_fin_temp1 = mapping_merge_df_final_grp_fin_temp1[mapping_merge_df_final_grp_fin_temp1['세부사업명_지역']==mapping_merge_df_final_grp_fin_temp1['납품요구건명_지역']]

In [53]:
# 최종필터링2 - 필수키워드 패턴 포함 & 지역 불일치 제거
mapping_merge_df_final_grp_fin_temp2 = mapping_merge_df_final_grp_fin[(mapping_merge_df_final_grp_fin['세부사업명_키워드2']!='')&(mapping_merge_df_final_grp_fin['납품요구건명_키워드2']!='')].reset_index(drop=True)

# 필요키워드 패턴 포함여부 확인
mapping_merge_df_final_grp_fin_temp2['세부사업명_키워드2'] = mapping_merge_df_final_grp_fin_temp2['세부사업명_키워드2'].str.split() ## 키워드별 정리
mapping_merge_df_final_grp_fin_temp2['납품요구건명_키워드2'] = mapping_merge_df_final_grp_fin_temp2['납품요구건명_키워드2'].str.split() ## 키워드별 정리

mapping_merge_df_final_grp_fin_temp2 = mapping_merge_df_final_grp_fin_temp2[mapping_merge_df_final_grp_fin_temp2.apply(check_inclusion2,axis=1)].reset_index(drop=True)

mapping_merge_df_final_grp_fin_temp2['세부사업명_키워드2'] = mapping_merge_df_final_grp_fin_temp2['세부사업명_키워드2'].apply(lambda x : ' '.join(sorted(set(x)))).str.strip()
mapping_merge_df_final_grp_fin_temp2['납품요구건명_키워드2'] = mapping_merge_df_final_grp_fin_temp2['납품요구건명_키워드2'].apply(lambda x : ' '.join(sorted(set(x)))).str.strip()

# 지역 불일치 제거
mapping_merge_df_final_grp_fin_temp2 = mapping_merge_df_final_grp_fin_temp2[mapping_merge_df_final_grp_fin_temp2['세부사업명_지역']==mapping_merge_df_final_grp_fin_temp2['납품요구건명_지역']]

In [73]:
# 최종 필터링 데이터셋
mapping_merge_df_final_grp_final = pd.concat([mapping_merge_df_final_grp_fin_temp1,mapping_merge_df_final_grp_fin_temp2]).drop_duplicates().reset_index(drop=True)
mapping_merge_df_final_grp_final.drop(columns = 'all_kwd', inplace=True)

In [65]:
# 내보내기
mapping_final_col = ['지역명', '자치단체명', '세부사업명', '수요기관명', '수요기관지역명','납품요구건명', 'all_kwd_fin']
mapping_merge_df_final_grp_final = mapping_merge_df_final_grp_final[mapping_final_col]
mapping_merge_df_final_grp_final.rename(columns={'all_kwd_fin':'키워드'},inplace=True)
mapping_merge_df_final_grp_final.to_csv('C:/py_src/midoproject/result/filtered_mapping_df.csv',encoding='utf-8-sig',index=False)

In [61]:
# except_mapping_list = ['고령우곡면게이트볼장시설물보수공사','용동공원 축구장 시설개선사업(전환사업)','해그늘 체육공원 족구장 개선','원미레포츠공원 축구장 개보수','대마산업단지 야구장 정비사업(2단계 도 전환사업)(전환사업)']

In [79]:
# test[~test['세부사업명'].isin(except_mapping_list)].reset_index(drop=True).to_csv('C:/py_src/midoproject/result/filtered_df_final.csv',encoding='utf-8-sig',index=False)

#### 교육청 예산서

In [388]:
# 종합쇼핑몰 학교 관련 사업
shop_edu_mapping_temp1 = all_shop_df_fin[all_shop_df_fin['납품요구건명'].str.contains('학교|교육|교육청|교육지원청|교육부|공립|사립|국립')]
shop_edu_mapping_temp2 = all_shop_df_fin[all_shop_df_fin['수요기관명'].str.contains('학교|교육|교육청|교육지원청|교육부|공립|사립|국립')]
shop_edu_mapping_df = pd.concat([shop_edu_mapping_temp1,shop_edu_mapping_temp2],axis=0).drop_duplicates().reset_index(drop=True)
shop_edu_mapping_df

,납품요구접수일자,수요기관명,납품요구건명,업체명,단가,단위,수량,금액,수요기관코드,수요기관구분,수요기관지역명,납품요구지청명,납품요구건명_re
0,2024-07-03,인천광역시남부교육청 인성초등학교,인성초등학교 운동장 인조잔디 교체,케이앤비준우(주),73000,㎡,788.0,57524000,7320148,기타기관,인천광역시 중구,인천지방조달청,인성 운동장 인조잔디 교체
1,2024-07-03,서울특별시동부교육청 상봉중학교,상봉중학교 운동장 환경개선관련 관급자재 구매,주식회사 액션필드,60700,㎡,2597.0,157637900,7020148,교육기관,서울특별시 중랑구,서울지방조달청,상봉 운동장 환경개선관련 관급자재 구매
2,2024-07-03,선린대학,"선린대학교 운동장 환경개선(인조잔디, 트랙 등) 공사 관급자재 발주",주식회사 네오필드,74500,㎡,4890.0,364305000,7003548,기타기관,경상북도 포항시 북구,대구지방조달청,선린대학교 운동장 환경개선인조잔디 트랙 등 공사 관급자재 발주
3,2024-07-02,경기도용인교육청 용동중학교,용동중학교 운동장개선 공사 인조잔디 구입,케이앤비준우(주),39500,㎡,3600.0,142200000,7750018,기타기관,경기도 용인시 처인구,인천지방조달청,용동 운동장개선 공사 인조잔디 구입
4,2024-07-01,충청남도교육청 충남삼성고등학교,충남삼성고등학교 운동장 인조잔디 구매,주식회사 플랜에이,63200,㎡,3070.0,194024000,8140366,기타기관,충청남도 아산시,대전지방조달청,충남삼성 운동장 인조잔디 구매
...,...,...,...,...,...,...,...,...,...,...,...,...,...
224,2024-01-09,서울특별시강동교육청 서울가동초등학교,실내운동장 인조잔디 구입,주식회사 필드글로벌,63000,㎡,495.0,31185000,7070052,교육기관,서울특별시 송파구,서울지방조달청,실내운동장 인조잔디 구입
225,2024-01-08,제주특별자치도교육청 제주시교육지원청 동광초등학교,동광초 인조잔디 및 울타리 조성공사 관급자재 구입(인조잔디 외 1종),주식회사 필드글로벌,63000,㎡,2680.0,168840000,9296060,교육기관,제주특별자치도 제주시,제주지방조달청,동광초 인조잔디 및 울타리 조성공사 관급자재 구입인조잔디 외 종
226,2024-01-04,한국과학기술원부설한국과학영재학교,풋살장 조성 및 기타공사 관급자재 구매,주식회사 이원,38000,㎡,747.0,28386000,B552516,기타공공기관,부산광역시 부산진구,부산지방조달청,풋살장 조성 및 기타공사 관급자재 구매
227,2024-01-03,경기도파주교육청 파주중학교,파주중 인조잔디 운동장 조성사업 관급자재 구입,주식회사 지에스케이,61500,㎡,2540.0,156210000,7680017,교육기관,경기도 파주시,서울지방조달청,파주중 인조잔디 운동장 조성사업 관급자재 구입


In [389]:
# 교육청, 교육지원청, 학교명 필터링
shop_edu_mapping_df['수요기관명_re'] = shop_edu_mapping_df['수요기관명'].str.replace('교육청','').str.replace('교육지원청','')
filter_edu1 = list(shop_edu_mapping_df[shop_edu_mapping_df['수요기관명_re'].str.split().str[0].notnull()]['수요기관명_re'].str.split().str[0].unique())
filter_edu2 = list(shop_edu_mapping_df[shop_edu_mapping_df['수요기관명_re'].str.split().str[1].notnull()]['수요기관명_re'].str.split().str[1].unique())
filter_edu3 = list(shop_edu_mapping_df[shop_edu_mapping_df['수요기관명_re'].str.split().str[2].notnull()]['수요기관명_re'].str.split().str[2].unique())

In [390]:
# 전체 키워드 데이터셋
edu_mapping_keywd_all = list(chain(*shop_edu_mapping_df['납품요구건명_re'].str.split(' '))) ## 전체 split 키워드
edu_mapping_keywd_df = pd.DataFrame(Counter(edu_mapping_keywd_all).items(),columns=['키워드','빈도수'])
edu_mapping_keywd_df = edu_mapping_keywd_df[edu_mapping_keywd_df['키워드'].apply(len)!=1].reset_index(drop=True)
edu_mapping_keywd_df.sort_values('빈도수',ascending=False)

,키워드,빈도수
2,인조잔디,130
1,운동장,84
15,구입,77
7,구매,64
55,관급자재인조잔디,63
...,...,...
164,토목공사,1
162,화산,1
161,목포부주,1
160,인조잔디구매,1


In [391]:
# 빅쿼리에서 불러오기
edu_budget_df = get_dataframe_from_bigquery('edu','edu_budget',bigquerykey_path)
edu_budget_df.sort_values('시도').reset_index(drop=True)

,시도,시군구,구분,과업명,금액,면적,예산집행
0,강원특별자치도,화천교육지원청,공립,상서중학교 운동장 보수,"89,533,000","4,664",학교
1,강원특별자치도,인제교육지원청,공립,상남중학교 개방형 운동장 조성,"725,302,000","6,507",교육청
2,강원특별자치도,강원특별자치도교육청,공립,강릉제일고등학교 축구장 인조잔디 교체,"1,020,560,000","8,576",교육청
3,강원특별자치도,강릉교육지원청,공립,관동중학교 인조잔디 운동장 조성,"806,400,000","4,082",교육청
4,강원특별자치도,강원특별자치도교육청,사립,동해광희고등학교 인조잔디운동장 보수,"51,700,000",None,None
...,...,...,...,...,...,...,...
189,충청남도,보령교육지원청,공립,성주초등학교 운동장 교체,"70,980,000",None,None
190,충청남도,당진교육지원청,공립,면천중학교 운동장 교체,"297,561,000",None,None
191,충청북도,충청북도교육청,공립,충북산업과학고등학교 다목적구장 조성,"769,799,000","5,540",교육청
192,충청북도,청주교육지원청,공립,서경중학교 운동장 보수 및 바닥교체,"602,871,000","3,500",교육청


In [392]:
# 전처리
edu_budget_df['과업명_re'] = edu_budget_df['과업명'].apply(clean_text)
edu_budget_df['과업명_re'] = edu_budget_df['과업명_re'].str.replace('초등학교','').str.replace('중학교','').str.replace('고등학교','')
edu_budget_df['과업명_re'] = edu_budget_df['과업명_re'].str.strip()

In [394]:
# 종합쇼핑몰 형식과 통일
edu_budget_df['시군구_re'] = np.where(edu_budget_df['시군구'].str.contains('교육지원청'),
                                   edu_budget_df['시도'] + edu_budget_df['시군구'].str.replace('교육지원청',''), edu_budget_df['시군구']) 

In [396]:
# 교육청, 교육지원청, 학교명 추출
filter_edu1_temp = pd.concat([edu_budget_df[edu_budget_df['시군구_re'].isin(filter_edu1)],
                              edu_budget_df[edu_budget_df['과업명'].str.contains(('|').join(filter_edu1))],
                              edu_budget_df[edu_budget_df['과업명_re'].str.contains(('|').join(filter_edu1))]],axis=0)
# edu_budget_df[edu_budget_df['시군구_re'].str.contains(('|').join(filter_edu1))].shape

filter_edu2_temp = pd.concat([edu_budget_df[edu_budget_df['시군구_re'].isin(filter_edu2)],
                              edu_budget_df[edu_budget_df['시군구_re'].str.contains(('|').join(filter_edu2))],
                              edu_budget_df[edu_budget_df['과업명'].str.contains(('|').join(filter_edu2))],
                              edu_budget_df[edu_budget_df['과업명_re'].str.contains(('|').join(filter_edu2))]],axis=0)

filter_edu3_temp = pd.concat([edu_budget_df[edu_budget_df['시군구_re'].isin(filter_edu3)],
                              edu_budget_df[edu_budget_df['시군구_re'].str.contains(('|').join(filter_edu3))],
                              edu_budget_df[edu_budget_df['과업명'].str.contains(('|').join(filter_edu3))],
                              edu_budget_df[edu_budget_df['과업명_re'].str.contains(('|').join(filter_edu3))]],axis=0)

filtered_edu_budget_df = pd.concat([filter_edu1_temp, filter_edu2_temp, filter_edu3_temp],axis=0).drop_duplicates().reset_index(drop=True)

In [406]:
# 종합쇼핑몰 학교 관련 키워드 빈도수
edu_filter_keywd = ('|').join(edu_mapping_keywd_df[edu_mapping_keywd_df['빈도수'] <= 3]['키워드'].unique())
edu_mapping_keywd_df[edu_mapping_keywd_df['빈도수'] <= 3].sort_values('빈도수',ascending=False) ## 빈도수 평균 2.99

,키워드,빈도수
43,다목적,3
25,운동장환경개선,3
3,교체,3
117,따른,3
47,물품,3
...,...,...
162,화산,1
161,목포부주,1
160,인조잔디구매,1
159,동북,1


In [420]:
# 교육청 제외 데이터
filtered_edu_budget_df_fin = filtered_edu_budget_df[filtered_edu_budget_df['과업명'].str.contains(edu_filter_keywd)].drop_duplicates().reset_index(drop=True)
filtered_edu_budget_df_fin

,시도,시군구,구분,과업명,금액,면적,예산집행,과업명_re,시군구_re
0,충청남도,보령교육지원청,공립,성주초등학교 운동장 교체,"70,980,000",None,None,성주 운동장 교체,충청남도보령
1,충청남도,보령교육지원청,공립,청파초등학교 운동장 교체,"20,976,000",None,None,청파 운동장 교체,충청남도보령
2,경기도,김포교육지원청,공립,신양중학교 친환경 운동장 조성 사업,None,None,None,신양 친환경 운동장 조성 사업,경기도김포
3,경기도,수원교육지원청,공립,수성중학교 친환경 운동장 조성 사업,None,None,None,수성 친환경 운동장 조성 사업,경기도수원
4,경기도,연천교육지원청,공립,백학중학교 친환경 운동장 조성 사업,None,None,None,백학 친환경 운동장 조성 사업,경기도연천
5,경기도,용인교육지원청,공립,어정초등학교 친환경 운동장 조성 사업,None,None,None,어정 친환경 운동장 조성 사업,경기도용인
6,경기도,용인교육지원청,공립,서원중학교 친환경 운동장 조성 사업,None,None,None,서원 친환경 운동장 조성 사업,경기도용인
7,경기도,파주교육지원청,공립,와석초등학교 친환경 운동장 조성 사업,None,None,None,와석 친환경 운동장 조성 사업,경기도파주
8,경기도,포천교육지원청,공립,영북초등학교 친환경 운동장 조성 사업,None,None,None,영북 친환경 운동장 조성 사업,경기도포천
9,서울특별시,동부교육지원청,공립,상봉중학교(운동장 환경 개선),"680,000,000","4,703",학교,상봉운동장 환경 개선,서울특별시동부


In [418]:
edu_budget_df[~edu_budget_df['과업명'].isin(filtered_edu_budget_df_fin['과업명'])].reset_index(drop=True)

,시도,시군구,구분,과업명,금액,면적,예산집행,과업명_re,시군구_re
0,전북특별자치도,전북특별자치도교육청,None,완주 스포츠클럽 실외야구장,"1,689,820,000",None,None,완주 스포츠클럽 실외야구장,전북특별자치도교육청
1,경기도,경기도교육청,None,친환경 운동장 조성사업비 지원 (30교),"15,000,000,000",None,학교,친환경 운동장 조성사업비 지원 교,경기도교육청
2,전라남도,전라남도교육청,None,운동장 개보수(고2),"1,652,038,000",None,학교,운동장 개보수고,전라남도교육청
3,울산광역시,울산광역시교육청,None,친환경운동장조성 다목적구장 중학교 1교,"14,000,000",None,학교,친환경운동장조성 다목적구장 교,울산광역시교육청
4,울산광역시,울산광역시교육청,None,친환경운동장조성 다목적구장 고등학교 6교,"465,600,000",None,학교,친환경운동장조성 다목적구장 교,울산광역시교육청
...,...,...,...,...,...,...,...,...,...
138,경상남도,의령교육지원청,사립,정곡중학교 체육관 신축(계속비),"38,000,000",None,학교,정곡 체육관 신축계속비,경상남도의령
139,대구광역시,대구광역시교육청,사립,대륜고등학교 축구장 인조잔디 재조성,"578,358,000","7,447",학교,대륜 축구장 인조잔디 재조성,대구광역시교육청
140,울산광역시,울산광역시교육청,사립,친환경운동장조성 다목적구장 중학교 (사립) 1교,"169,141,500",None,학교,친환경운동장조성 다목적구장 사립 교,울산광역시교육청
141,전북특별자치도,전북특별자치도교육청,사립,익산고등학교 인조잔디 조성,"1,050,001,000","7,417",교육청,익산 인조잔디 조성,전북특별자치도교육청
